In [5]:
import numpy as np
import pandas as pd
def DataGenerator(filename = 'Train_Dst_NoAuction_ZScore_CF_1.txt',test = False):
    with open('Train_Dst_NoAuction_ZScore_CF_2.txt') as f:
        test_size = 4000
        if test:
            new = np.array(f.readline().split('  ')[1:],dtype='float64')[:,np.newaxis][:test_size]
        else:
            new = np.array(f.readline().split('  ')[1:],dtype='float64')[:,np.newaxis]
        hi = True
        count = 0
        while hi:
            try:
                if test:
                    b = np.array(f.readline().split('  ')[1:],dtype='float64')[:,np.newaxis][:test_size]
                else:
                    b = np.array(f.readline().split('  ')[1:],dtype='float64')[:,np.newaxis]
                new = np.concatenate((new,b),axis=1)
                count += 1
            except:
                hi = False
        f.close()
    new = pd.DataFrame(new)
    return new

In [6]:
def Preprocess(data = None,k = 5):
    """
    Parameter k: k=1,2,3,5,10
    """
    m,n = data.shape
    bid = data.loc[:,2:39:4]
    ask = data.loc[:,0:39:4]
    vol_a = data.loc[:,1:39:4]
    vol_b = data.loc[:,3:39:4]
    ask.columns = vol_a.columns
    bid.columns = vol_b.columns
    vol_a.sort_values(list(vol_a.index.values),axis=1,ascending = True,inplace = True)
    vol_b.sort_values(list(vol_b.index.values),axis=1,ascending = False,inplace = True)
    ask = ask.loc[:,vol_a.columns]
    bid = bid.loc[:,vol_b.columns]
    mid = (bid.iloc[:,0]+ask.iloc[:,0])/2
    mid_change = mid/mid.shift(1)-1

    ask_column = []
    bid_column = []
    ask_vol_column = []
    bid_vol_column = []
    for i in range(10):
        ask_column.append('a_'+str(i))
        bid_column.append('b_'+str(i))
        ask_vol_column.append('va_'+str(i))
        bid_vol_column.append('vb_'+str(i))
    mid_column = ['mid','mc']

    result = pd.concat([ask,bid,vol_a,vol_b,mid,mid_change],axis=1)
    result.columns = np.concatenate([ask_column,bid_column,ask_vol_column,bid_vol_column,mid_column])

    result.iloc[:,:20]  =result.iloc[:,:20].values / result['mid'].values[:,np.newaxis]-1
    for i in range(9):
        result.iloc[:,i+21] += result.iloc[:,i+20]
        result.iloc[:,i+31] += result.iloc[:,i+30]
    
    result = result.iloc[1:]
    
    labels = [1,2,3,5,10]
    label  = data.iloc[:,labels.index(k)-5]
    
    return result,label

In [25]:
data = DataGenerator(test =True)
data,label = Preprocess(data)
label = label-1
label = np_utils.to_categorical(label,num_classes = 3)

//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [26]:
label.shape

(4000, 3)

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Softmax
from keras.layers import LSTM,Conv1D,Dropout,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils

In [3]:
#Convolution
kernel_size = 5
pool_size = 2
time_step = 300
num_feature = 42
#LSTM
lstm_output_size = 32
batch_size = 32
epochs = 10

print('Build model...')
model = Sequential()
model.add(Conv1D(filters=16,input_shape = (time_step,num_feature),
                kernel_size = kernel_size,
                padding = 'causal',
                activation = 'relu',
                strides = 1))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(filters=16,
                kernel_size = kernel_size,
                padding = 'causal',
                activation = 'relu',
                strides = 1))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(filters=32,
                kernel_size = kernel_size,
                padding = 'causal',
                activation = 'relu',
                strides = 1))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(filters=32,
                kernel_size = kernel_size,
                padding = 'causal',
                activation = 'relu',
                strides = 1))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(LSTM(lstm_output_size))
model.add(Dense(units = 64,
               activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 3))
model.add(Activation('softmax'))


rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

print('Train')
model.fit(x_train,y_train,
          batch_size = batch_size,
          epochs = epochs,
          validation_data = (x_test,y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test cost:', score)
print('Test accuracy:', acc)

Build model...
Train


NameError: name 'x_train' is not defined

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 300, 16)           3376      
_________________________________________________________________
batch_normalization_1 (Batch (None, 300, 16)           64        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 150, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 150, 16)           1296      
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 16)           64        
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 75, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 75, 32)            2592      
__________